# ✅  1 — Objectifs

Ce notebook a pour objectifs :

1. Créer la base DuckDB `energie_logement.duckdb` dans le dossier `01_import`
2. Charger les données DPE depuis les parquets (`parquet_dpe/`)
3. Charger les données Enedis depuis les parquets (`parquet_enedis/`)
4. Vérifier la bonne intégration des données (volumétrie, schéma)

# ✅ 2 — Imports & chemins

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from config import *

PROJECT_ROOT = Path.cwd().parent




# ✅ 3 — Création / ouverture de la base DuckDB

In [3]:
con = duckdb.connect(str(DB_PATH))
con.execute(f"PRAGMA memory_limit='{DUCKDB_MEMORY_LIMIT}';")
con.execute(f"PRAGMA threads={DUCKDB_THREADS};")
con.execute("PRAGMA enable_progress_bar=false;")
con.execute("PRAGMA disable_object_cache;")
con.execute("PRAGMA preserve_insertion_order=false;")

print("✅ Connecté à DuckDB")

✅ Connecté à DuckDB


# ✅ 4 — Fonctions de checkpoint du chargement des données dans DuckDB

In [4]:
def load_checkpoint(path: Path):
    if path.exists():
        with open(path, "r") as f:
            return json.load(f)
    return {"next_index": 0}

def save_checkpoint(path: Path, next_index: int):
    with open(path, "w") as f:
        json.dump({"next_index": int(next_index)}, f, indent=2)

def list_parquet_files(folder: Path, pattern: str):
    files = sorted(folder.glob(pattern))
    return [str(f) for f in files]

# ✅ 5 — Fonctions utilitaires

In [5]:
def get_table_cols(con, table_name: str):
    df = con.execute(f"PRAGMA table_info('{table_name}')").df()
    return df["name"].tolist()

def get_batch_cols(con, files):
    df = con.execute("""
        DESCRIBE
        SELECT *
        FROM read_parquet(?, union_by_name=true)
    """, [files]).df()
    return set(df["column_name"])

In [6]:
def ingest_parquet_folder_with_checkpoint(
    con,
    table_name: str,
    parquet_files: list,
    checkpoint_path: Path,
    batch_size: int,
    drop_if_start_from_zero: bool = True,
):
    """
    Ingestion incrémentale de fichiers parquet dans DuckDB.
    - checkpoint: index du prochain fichier à charger
    - schema évolutif: ajoute des colonnes VARCHAR si apparaissent dans les nouveaux batches
    """
    checkpoint = load_checkpoint(checkpoint_path)
    start = int(checkpoint.get("next_index", 0))

    if start == 0 and drop_if_start_from_zero:
        print(f"🆕 Démarrage depuis zéro → DROP TABLE IF EXISTS {table_name}")
        con.execute(f"DROP TABLE IF EXISTS {table_name}")
    else:
        print(f"🔁 Reprise → table {table_name} conservée, reprise à l'index: {start}")

    t_global_start = time.time()

    if len(parquet_files) == 0:
        print(f"⚠️ Aucun fichier parquet trouvé pour {table_name}.")
        return

    for i in range(start, len(parquet_files), batch_size):
        t_batch_start = time.time()
        batch = parquet_files[i:i + batch_size]

        # Création initiale
        if start == 0 and i == 0:
            con.execute(f"""
                CREATE TABLE {table_name} AS
                SELECT *
                FROM read_parquet(?, union_by_name=true)
            """, [batch])

        else:
            # 1) colonnes table existante + colonnes batch
            table_cols = get_table_cols(con, table_name)   # list
            batch_cols = get_batch_cols(con, batch)        # set

            # 2) nouvelles colonnes
            new_cols = sorted(batch_cols - set(table_cols))
            if new_cols:
                print(f"⚠️ {table_name}: {len(new_cols)} nouvelles colonnes → ALTER TABLE")
                for c in new_cols:
                    con.execute(f'ALTER TABLE {table_name} ADD COLUMN "{c}" VARCHAR')
                table_cols = get_table_cols(con, table_name)

            # 3) insert aligné
            insert_cols_sql = ", ".join(f'"{c}"' for c in table_cols)

            select_exprs = []
            for c in table_cols:
                if c in batch_cols:
                    select_exprs.append(f'"{c}"')
                else:
                    select_exprs.append(f'NULL AS "{c}"')

            select_sql = ",\n                ".join(select_exprs)

            insert_sql = f"""
                INSERT INTO {table_name} ({insert_cols_sql})
                SELECT
                    {select_sql}
                FROM read_parquet(?, union_by_name=true)
            """
            con.execute(insert_sql, [batch])

        # checkpoint = next index
        next_i = i + batch_size
        save_checkpoint(checkpoint_path, next_i)

        # log
        t_batch_end = time.time()
        batch_time = t_batch_end - t_batch_start
        speed = len(batch) / (batch_time / 60) if batch_time > 0 else float("inf")

        print(
            f"✅ {table_name}: fichiers {i} → {min(next_i, len(parquet_files))}/{len(parquet_files)} | "
            f"{speed:.1f} fichiers/min"
        )

    t_global_end = time.time()
    print(f"🎉 Chargement {table_name} terminé en {t_global_end - t_global_start:.1f}s")

# ✅ 6 — Chargement des DPE dans DuckDB

In [6]:
parquets_dpe = list_parquet_files(PARQUET_DPE, "dpe_part_*.parquet")
print("📦 Fichiers DPE:", len(parquets_dpe))

ingest_parquet_folder_with_checkpoint(
    con=con,
    table_name="dpe",
    parquet_files=parquets_dpe,
    checkpoint_path=CHECKPOINT_LOAD_DPE_IN_DUCKDB,
    batch_size=BATCH_SIZE_DUCKDB_DPE,
    drop_if_start_from_zero=True
)

📦 Fichiers DPE: 36
🆕 Démarrage depuis zéro → DROP TABLE IF EXISTS dpe
✅ dpe: fichiers 0 → 36/36 | 2481.3 fichiers/min
🎉 Chargement dpe terminé en 0.9s


# ✅ 7 — Chargement des données ENEDIS dans DuckDB

In [6]:
parquets_enedis = list_parquet_files(PARQUET_ENEDIS, "enedis_part_*.parquet")
print("📦 Fichiers ENEDIS:", len(parquets_enedis))

ingest_parquet_folder_with_checkpoint(
    con=con,
    table_name="enedis",
    parquet_files=parquets_enedis,
    checkpoint_path=CHECKPOINT_LOAD_ENEDIS_IN_DUCKDB,
    batch_size=BATCH_SIZE_DUCKDB_ENEDIS,
    drop_if_start_from_zero=True
)

📦 Fichiers ENEDIS: 33297
🔁 Reprise → table enedis conservée, reprise à l'index: 21200
✅ enedis: fichiers 21200 → 21400/33297 | 16440.7 fichiers/min
✅ enedis: fichiers 21400 → 21600/33297 | 20819.1 fichiers/min
✅ enedis: fichiers 21600 → 21800/33297 | 18492.1 fichiers/min
✅ enedis: fichiers 21800 → 22000/33297 | 18404.2 fichiers/min
✅ enedis: fichiers 22000 → 22200/33297 | 12659.0 fichiers/min
✅ enedis: fichiers 22200 → 22400/33297 | 11059.1 fichiers/min
✅ enedis: fichiers 22400 → 22600/33297 | 16702.1 fichiers/min
✅ enedis: fichiers 22600 → 22800/33297 | 15135.2 fichiers/min
✅ enedis: fichiers 22800 → 23000/33297 | 10537.3 fichiers/min
✅ enedis: fichiers 23000 → 23200/33297 | 16135.8 fichiers/min
✅ enedis: fichiers 23200 → 23400/33297 | 2341.2 fichiers/min
✅ enedis: fichiers 23400 → 23600/33297 | 19299.2 fichiers/min
✅ enedis: fichiers 23600 → 23800/33297 | 17179.8 fichiers/min
✅ enedis: fichiers 23800 → 24000/33297 | 18817.1 fichiers/min
✅ enedis: fichiers 24000 → 24200/33297 | 14982.

# ✅ 8 — Vérification des tables

In [6]:
tables = con.execute("SHOW TABLES").df()
print("Tables dans la base :")
display(tables)

for table in tables["name"]:
    n = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table} : {n} lignes")

Tables dans la base :


,name
0,dpe
1,dpe_adresse
2,dpe_adresse_harmonisee
3,dpe_ban_par_join_key
4,dpe_features_par_join_key
5,dpe_final_par_join_key
6,dpe_insee_norm
7,dpe_keyed
8,dpe_modes_par_adresse
9,enedis


dpe : 17343297 lignes
dpe_adresse : 17343297 lignes
dpe_adresse_harmonisee : 17343297 lignes
dpe_ban_par_join_key : 4303906 lignes
dpe_features_par_join_key : 4303907 lignes


RuntimeError: Query interrupted

# ✅ 9 — Fermeture de la connexion

In [7]:
# Fermer la connexion à DuckDB
con.close()
print("Connexion à DuckDB fermée.")


Connexion à DuckDB fermée.
